### Plot data from the netCDF conversions of the original log files and compare with SSDS generated version
To run this Notebook:
- Install Python3.8 (e.g from [Anacaonda](https://www.anaconda.com/distribution/))
- `git clone https://<your_account>@bitbucket.org/mbari/auv-python.git` Replacing: `<your_account>`
- `cd auv-python`
- `jupyter notebook`
- Select this file and execute the cells

In [ ]:
import xarray as xr
import hvplot.pandas
import ipywidgets as widgets
from os.path import join
from logs2netcdfs import BASE_PATH
from logs2netcdfs import LOG_FILES

# Enter vehicle and mission name here
auv_name = 'Dorado389'
mission = '2020.064.10'

# Compare SSDS generated with logs2netcdfs.py generated
ssds_path = join('http://dods.mbari.org:80/opendap/data/ssdsdata/ssds/generated/netcdf/files',
                 'ssds.shore.mbari.org/auvctd/missionlogs/', mission.split('.')[0],
                 ''.join(mission.split('.')[:2]), mission)

netcdfs_dir = join(BASE_PATH, auv_name, 'missionnetcdfs', mission)

Read all the netCDF file and SSDS OPeNDAP data into in-memory xarray objects saved in dictionaries

In [ ]:
ncs = {}
dds = {}
for lf in LOG_FILES:
    nc_file = lf.replace('.log', '.nc')
    ncs[nc_file] = xr.open_dataset(join(netcdfs_dir, nc_file))
    dds[nc_file] = xr.open_dataset(join(ssds_path, nc_file))

Show all the attributes of each data set and all the variables in each data set

In [ ]:
for nc_file, ds in ncs.items():
    print(nc_file)
    ##display(ds)  # Uncomment to explore all the attributes

Pick the converted log to examine

In [ ]:
ds_picker = widgets.Dropdown(options=ncs.keys())
display(ds_picker)

Pick the variable to plot

In [ ]:
var_picker = widgets.Dropdown(options=list(ncs[ds_picker.value]))
display(var_picker)

Make two plots comparing the locally generated netCDF file with the one created and served by SSDS

In [ ]:
from pandas.plotting import register_matplotlib_converters
static = False   # Set to True for non-interactive Matplotlib plots

register_matplotlib_converters()
local_ds = ncs[ds_picker.value]
ssds_ds = dds[ds_picker.value]

variable = var_picker.value

var = getattr(local_ds, variable)
try:
    d_var = getattr(ssds_ds, variable)
except AttributeError as e:
    d_var = xr.DataArray([])
    
if static:
    print(f"ds.{variable}.plot(aspect=5, size=5)")
    display(var.plot(aspect=5, size=5, title=f"Local netCDF file: {ds_picker.value}"))
    print("SSDS original data")
    display(d_var.plot(aspect=5, size=5, title=f"SSDS original data: {ds_picker.value}"))
else:
    yl = f"{var.attrs['long_name']} [{var.attrs.get('units', '')}]"
    print(f"ds.{variable}.to_pandas().hvplot(ylabel='{yl}')")
    display(var.to_pandas().hvplot(ylabel=yl, title=f"Local netCDF file: {ds_picker.value}"))
    if d_var.any():
        display(d_var.to_pandas().hvplot(ylabel='SSDS '+yl, title=f"SSDS original data: {ds_picker.value}"))
    else:
        display(f'SSDS Dataset has no variable {variable}')